### Diversity Measure using HHI Score
- Ref: diff_in_diff/HHI_modified.ipynb

In [1]:
# Import Modules
import pandas as pd
import numpy as np
import sqlite3
import pandas as pd
from nltk import FreqDist
import pickle
import math
import matplotlib.pyplot as plt
import statsmodels.formula.api as sm

# Import Dataset
conn = sqlite3.connect('/data1/StackOverflow/stackexchange-to-sqlite/stack.db')
query = '''
SELECT creation_date, tags
FROM questions
WHERE creation_date > '2020-11-30';
'''
df_tags = pd.read_sql_query(query, conn)
conn.close()

### 1) Preprocessing

In [2]:
def wc(text):
    """
    Cleaning function to be used with our first wordcloud
    """
    
    if text:
        tags = text.replace('><',' ')
        tags = tags.replace('-','')
        tags = tags.replace('.','DOT')
        tags = tags.replace('c++','Cpp')
        tags = tags.replace('c#','Csharp')
        tags = tags.replace('>','')
        return tags.replace('<','')
    else:
        return 'None'
    
def clean_tags(text):
    """
    Cleaning function for tags
    """
    
    if text:
        tags = text.replace('><',' ')
        tags = tags.replace('>','')
        return tags.replace('<','')
    else:
        return 'None'
    
def tag_freq(data):
    tags = data['tags'].str.replace('[\["\]]', '', regex=True)
    tags = [tag for i in tags.apply(lambda x: wc(x)) for tag in i.split(', ')]
    result = FreqDist(tags)
    return result

df_tags['creation_date'] = pd.to_datetime(df_tags['creation_date'])
df_tags['year_month'] = df_tags['creation_date'].dt.to_period('D')
df_tags['year_month'] = df_tags['year_month'].astype(str)
year_month = df_tags.year_month.unique()

In [4]:
# Extract keys througout the whole data
all_keys = tag_freq(df_tags)
all_keys = pd.DataFrame(all_keys, index = ['tag']).transpose().reset_index()

# compute tagShare on each month
for i in range(len(year_month)):
    data = df_tags[df_tags['year_month'] == year_month[i]]
    tags = tag_freq(data)
    tagCount = pd.DataFrame(tags, index = ['tag']).transpose().reset_index()
    tagShare = []
    for j in range(len(tagCount)):
        tagShare.append((tagCount['tag'][j] / tagCount['tag'].sum())*100)
    tagCount['tagShare'] = tagShare
    varName = year_month[i].replace('-', '_')
    tagCount = tagCount.rename(columns = {'tag':f'tag_{varName}','tagShare':f'tagShare_{varName}'})
    # merge here.
    all_keys = pd.merge(all_keys, tagCount, on = 'index', how = 'left')

### 2) Calculate HHI Index

In [5]:
# Define square_sum function
def square_sum(numbers):
    result = sum(x**2 for x in numbers if not math.isnan(x))
    return result
# Measure score
HHI_Score = []
# Calculate HHI for each monthly tag share column.
for i in range(3, all_keys.shape[1], 2):
    HHI_Score.append(square_sum(all_keys.iloc[:, i]))
result_HHI = pd.DataFrame({'year_month':year_month, 'HHI_Score':HHI_Score})

### 3) Diff-in-Diff
- Treated Group : 2022-09 ~ 2023-08 (12 months)
- Control Group : 2021-09 ~ 2022-08 (12 months)
- Diversity Ratio == HHI_Score

In [6]:
hhi = list(result_HHI[(result_HHI['year_month'] > '2021-08-31') &
           (result_HHI['year_month'] < '2023-09-01')].reset_index().HHI_Score) # fixed datetime
# Split Data
control_data = pd.DataFrame({'HHI' : hhi[:365],
              'T_d': [0]*len(hhi[:365]),
              'P_t' : [0]*90 + [1]*275})
treated_data = pd.DataFrame({'HHI' : hhi[365:],
              'T_d': [1]*len(hhi[365:]),
              'P_t' : [0]*90 + [1]*275})
df_did = pd.concat([control_data, treated_data], axis = 0).reset_index(drop = True)
# Add date and month feature
df_did['date'] = result_HHI[(result_HHI['year_month'] > '2021-08-31') &
           (result_HHI['year_month'] < '2023-09-01')].reset_index().year_month
df_did['month'] = pd.to_datetime(df_did['date']).dt.month
# Apply log
df_did['ln_y'] = np.log(df_did['HHI'])

### 4) Model Fitting

In [7]:
sm.ols('ln_y ~ T_d + P_t + T_d * P_t + C(month)', df_did).fit().summary().tables[1]

,coef,std err,t,P>|t|,[0.025,0.975]
Intercept,4.3000,0.073,59.097,0.000,4.157,4.443
C(month)[T.2],-0.0276,0.018,-1.510,0.131,-0.063,0.008
C(month)[T.3],-0.0524,0.018,-2.941,0.003,-0.087,-0.017
C(month)[T.4],-0.0175,0.018,-0.975,0.330,-0.053,0.018
C(month)[T.5],-0.0532,0.018,-2.989,0.003,-0.088,-0.018
C(month)[T.6],-0.0723,0.018,-4.028,0.000,-0.108,-0.037
C(month)[T.7],-0.0578,0.018,-3.248,0.001,-0.093,-0.023
C(month)[T.8],-0.0933,0.018,-5.240,0.000,-0.128,-0.058
C(month)[T.9],0.0294,0.074,0.400,0.690,-0.115,0.174
C(month)[T.10],0.0550,0.073,0.748,0.455,-0.089,0.199


In [8]:
sm.ols('ln_y ~ T_d + P_t + T_d * P_t + C(month)', df_did).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   ln_y   R-squared:                       0.692
Model:                            OLS   Adj. R-squared:                  0.686
Method:                 Least Squares   F-statistic:                     114.8
Date:                Mon, 15 Apr 2024   Prob (F-statistic):          4.44e-172
Time:                        13:15:39   Log-Likelihood:                 659.13
No. Observations:                 730   AIC:                            -1288.
Df Residuals:                     715   BIC:                            -1219.
Df Model:                          14                                         
Covariance Type:            nonrobust                                         
==================================================================================
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept          4.3000      0.073     59.097      0.000       4.157       4.443
C(month)[T.2]     -0.0276      0.018     -1.510      0.131      -0.063       0.008
C(month)[T.3]     -0.0524      0.018     -2.941      0.003      -0.087      -0.017
C(month)[T.4]     -0.0175      0.018     -0.975      0.330      -0.053       0.018
C(month)[T.5]     -0.0532      0.018     -2.989      0.003      -0.088      -0.018
C(month)[T.6]     -0.0723      0.018     -4.028      0.000      -0.108      -0.037
C(month)[T.7]     -0.0578      0.018     -3.248      0.001      -0.093      -0.023
C(month)[T.8]     -0.0933      0.018     -5.240      0.000      -0.128      -0.058
C(month)[T.9]      0.0294      0.074      0.400      0.690      -0.115       0.174
C(month)[T.10]     0.0550      0.073      0.748      0.455      -0.089       0.199
C(month)[T.11]     0.0186      0.071      0.261      0.794      -0.121       0.158
C(month)[T.12]     0.0515      0.018      2.893      0.004       0.017       0.086
T_d               -0.0903      0.015     -6.112      0.000      -0.119      -0.061
P_t                0.0702      0.072      0.978      0.328      -0.071       0.211
T_d:P_t           -0.2156      0.017    -12.664      0.000      -0.249      -0.182
==============================================================================
Omnibus:                       49.969   Durbin-Watson:                   1.190
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               49.828
Skew:                           0.591   Prob(JB):                     1.51e-11
Kurtosis:                       2.508   Cond. No.                         65.2
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [ ]:
# Save it to csv for Stata usage
#df_did.to_csv('HHI_did_modified.csv', index=False)
# df = pd.read_csv('HHI_did_modified.csv')